## Import Libraries

In [1]:
from neo4j import GraphDatabase
from pyvis.network import Network
import os
import webbrowser

## Data Loading

In [2]:
# Load config from file
def load_neo4j_config(filepath):
    config = {}
    with open(filepath, 'r') as file:
        for line in file:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key] = value
    return config

In [3]:
# Load credentials from the file
config = load_neo4j_config("Encryption/Stack Overflow Sandbox Credentials.txt")

In [4]:
# Assign values
uri = config.get("NEO4J_URI")
username = config.get("NEO4J_USERNAME")
password = config.get("NEO4J_PASSWORD")
database = config.get("NEO4J_DATABASE", "neo4j")

## Initialise Data Driver

In [5]:
# Create driver
driver = GraphDatabase.driver(uri, auth=(username, password))

In [6]:
# Example query to test
def test_connection():
    with driver.session() as session:
        result = session.run("RETURN 'Connection successful!' AS message")
        print(result.single()["message"])

In [7]:
test_connection()

Connection successful!


## Example 1

In [8]:
# Function to run your Cypher query with a parameter
def get_top_answerers_by_tag(tag_name):
    query = """
    MATCH (t:Tag {name:$tagName})<-[:TAGGED]-(q:Question)<-[:ANSWERED]-(a:Answer {is_accepted:true})<-[:PROVIDED]-(u:User)
    RETURN DISTINCT u.display_name as answerer
    LIMIT 5
    """
    with driver.session(database=database) as session:
        result = session.run(query, tagName=tag_name)
        return [record["answerer"] for record in result]

In [9]:
# Example usage
top_answerers = get_top_answerers_by_tag("python")
print("Top Python Answerers:")
for name in top_answerers:
    print("-", name)

Top Python Answerers:
- T. de Jong
- Abhishek Tyagi
- cybersam
- jose_bacoy
- Thomas J


## Example 2

In [10]:
def run_graph_query(tag_name):
    query = """
    MATCH (t:Tag {name:$tagName})<-[:TAGGED]-(q:Question)<-[:ANSWERED]-(a:Answer {is_accepted:true})<-[:PROVIDED]-(u:User)
    RETURN t, q, a, u
    LIMIT 25
    """
    with driver.session(database=database) as session:
        result = session.run(query, tagName=tag_name)
        return list(result)  # 👈 CONVERT TO LIST HERE

In [11]:
def visualise_graph(result):
    os.makedirs("Output", exist_ok=True)

    net = Network(notebook=False, height="750px", width="100%", directed=True)
    nodes = set()

    for record in result:
        t = record.get("t")
        q = record.get("q")
        a = record.get("a")
        u = record.get("u")

        # Safely handle nodes
        for node, label in zip([t, q, a, u], ['Tag', 'Question', 'Answer', 'User']):
            if node and node.id not in nodes:
                node_id = node.id or str(uuid.uuid4())
                node_label = node.get('name') or node.get('title') or node.get('display_name') or label
                net.add_node(node_id, label=node_label, title=label)
                nodes.add(node_id)

        # Add edges (with safety)
        if t and q:
            net.add_edge(t.id, q.id, label="TAGGED")
        if q and a:
            net.add_edge(q.id, a.id, label="ANSWERED")
        if a and u:
            net.add_edge(a.id, u.id, label="PROVIDED")

    # net.show("output/neo4j_graph_example_2.html", notebook=False)
    
    abs_path = os.path.abspath("output/neo4j_graph_example_2.html")
    webbrowser.open('file://' + abs_path)

In [12]:
# Run and visualise
result = run_graph_query("neo4j")
visualise_graph(result)

C:\Users\HaochengLin\AppData\Local\Temp\ipykernel_22896\2459234817.py:15: DeprecationWarning: `id` is deprecated, use `element_id` instead
  if node and node.id not in nodes:
C:\Users\HaochengLin\AppData\Local\Temp\ipykernel_22896\2459234817.py:16: DeprecationWarning: `id` is deprecated, use `element_id` instead
  node_id = node.id or str(uuid.uuid4())
C:\Users\HaochengLin\AppData\Local\Temp\ipykernel_22896\2459234817.py:23: DeprecationWarning: `id` is deprecated, use `element_id` instead
  net.add_edge(t.id, q.id, label="TAGGED")
C:\Users\HaochengLin\AppData\Local\Temp\ipykernel_22896\2459234817.py:25: DeprecationWarning: `id` is deprecated, use `element_id` instead
  net.add_edge(q.id, a.id, label="ANSWERED")
C:\Users\HaochengLin\AppData\Local\Temp\ipykernel_22896\2459234817.py:27: DeprecationWarning: `id` is deprecated, use `element_id` instead
  net.add_edge(a.id, u.id, label="PROVIDED")
